First, import numpy and scipy library.

In [22]:
import numpy as np
import scipy.optimize as sopt

Define the cost function $t(x)$ and $xt(x)$, which is the objective function of SO.
list the time cost array of the flow from vertex 1 and 2 separately. 

In [23]:
def t(x):
    x13, x23, x14, x24, x34_1, x34_2 = x
    x34 = x34_1+x34_2
    t13 = 0.5+0.001*x13
    t23 = 0.5+0.001*x23
    t14 = 1.0+0.0005*x14
    t24 = 1.0+0.001*x24
    t34 = 1.0+0.002*x34
    t_array = [t13, t23, t14, t24, t34] 
    t_source_1 = [t14, t13+t34]
    t_source_2 = [t24, t23+t34]
    return t_array, t_source_1, t_source_2

def sum_tilde_t(x):
    x13, x23, x14, x24, x34_1, x34_2 = x
    x34 = x34_1+x34_2
    tilde_t13 = 0.5*x13+0.001*x13**2
    tilde_t23 = 0.5*x23+0.001*x23**2
    tilde_t14 = 1.0*x14+0.0005*x14**2
    tilde_t24 = 1.0*x24+0.001*x24**2
    tilde_t34 = 1.0*x34+0.002*x34**2
    return tilde_t13 + tilde_t23 + tilde_t14 + tilde_t24 + tilde_t34

 # calculate next point based on step size alpha, current point x and direction y
def cal_next_guess(alpha, x, y1, y2):
    alpha1 = alpha[0]
    alpha2 = alpha[1]
    x13, x23, x14, x24, x34_1, x34_2 = x
    y14 = y1[0]
    y134 = y1[1]
    y24 = y2[0]
    y234 = y2[1]

    x14 += alpha1*(y14-x14)
    x13 += alpha1*(y134-x13)
    x24 += alpha2*(y24-x24)
    x23 += alpha2*(y234-x23)
    x34_1 += alpha1*(y134-x34_1)
    x34_2 += alpha2*(y234-x34_2)

    next_point_x = [x13, x23, x14, x24, x34_1, x34_2]
    return next_point_x

Initialize the set of points as the list named guesses. Define the total flow in three routes. 

In [24]:
guesses = []
total_demand_1 = 2000
total_demand_2 = 1000

Function $step$: contain task for one iteration.
1. Define and calulate cost $t(x)$ and objective function $sum\_tilde\_t(x)$.
2. Find drection $y$: Set the flow direction $y$ to the route with minimum cost. 
3. Find the step size $\alpha \in (0,1)$ 
    1. Define the objective function after moving as $func$.
    2. Define the constraints $con$ to satisfy Total flow conservation and bound $\alpha \in (0,1)$.
    3. Use $scipy.optimize.minimize$ tool to obtain the value of step size $\alpha$.
5. Update flow as $x^{n+1} = x^n + \alpha * (y^n-x^n)$
6. Do convergence test. If satisfy the convergence test, stop the iteration; if not, enter next iteration. 


In [25]:
def step(i):
    print('in step',i+1)
    
    x = guesses[-1]

    t_array, t_source_1, t_source_2 = t(x)
    
    Z = sum_tilde_t(x)
    
    print('\tUpdate \t\tt\t', np.round(t_array,decimals=3), '\tZ(x)\t', np.round(Z,decimals=2))
    print('\tUpdate \t\tt\t', np.round(t_source_1,decimals=3), np.round(t_source_2,decimals=3))
    
    # find y1 with min t in from source 1 
    # t_source_1 = [t14, t13+t34]
    y1_index = np.argmin(t_source_1)
    y1 = np.array([0.0, 0.0])
    y1[y1_index] = total_demand_1
    print('\tDirection \ty1\t', np.round(y1,decimals=2))

    # find y2 with min t in from source 2
    # t_source_2 = [t24, t23+t34]
    y2_index = np.argmin(t_source_2)
    y2 = np.array([0.0, 0.0])
    y2[y2_index] = total_demand_2
    print('\tDirection \ty2\t', np.round(y2,decimals=2))

    # objective function to minimize
    def func(alpha):
        next_point_x = cal_next_guess(alpha,x,y1,y2)
        return sum_tilde_t(next_point_x)

    # constrain: sum of flow equals to total flow
    # def con(alpha):
    #     return np.sum(x+alpha*(y-x)) - tot_flow
    
    # cons = {'type':'eq', 'fun': con}
    
    # find alpha that minimize objective function
    alpha = np.array([0.0, 0.0])
    bounds = [(0,1) for i in range(2)]
    res = sopt.minimize(func, alpha, bounds=bounds)
    alpha = res.x
    print('\tStep \t\talpha\t', np.round(alpha,decimals=3))

    # update x
    next_guess = cal_next_guess(alpha,x,y1,y2)
    guesses.append(next_guess)
    print('\tMove \t\tx\t', np.round(next_guess,decimals=3))
    

main function to solve the Frank Wolfe for SO problem. 

In [26]:
if __name__ == '__main__':
    # x13, x23, x14, x24, x34_1, x34_2 = x
    guesses = [np.array([0, 0, 2000, 1000, 0, 0])]
    print('Init Guesses\t\tx\t',guesses[0])

    for i in range(2):
        step(i)
    

Init Guesses		x	 [   0    0 2000 1000    0    0]
in step 1
	Update 		t	 [0.5 0.5 2.  2.  1. ] 	Z(x)	 6000.0
	Update 		t	 [2.  1.5] [2.  1.5]
	Direction 	y1	 [   0. 2000.]
	Direction 	y2	 [   0. 1000.]
	Step 		alpha	 [0.075 0.113]
	Move 		x	 [ 150.   112.5 1850.   887.5  150.   112.5]
in step 2
	Update 		t	 [0.65  0.613 1.925 1.887 1.525] 	Z(x)	 5803.12
	Update 		t	 [1.925 2.175] [1.887 2.137]
	Direction 	y1	 [2000.    0.]
	Direction 	y2	 [1000.    0.]
	Step 		alpha	 [0. 0.]
	Move 		x	 [ 150.   112.5 1850.   887.5  150.   112.5]
